In [1]:
!pip install mediapipe opencv-python

  Using cached matplotlib-3.7.4-cp38-cp38-win_amd64.whl.metadata (5.8 kB)
     ---------------------------------------- 0.0/904.4 kB ? eta -:--:--
     ---- --------------------------------- 112.6/904.4 kB 2.2 MB/s eta 0:00:01
     ---------------------- --------------- 532.5/904.4 kB 5.6 MB/s eta 0:00:01
     -------------------------------------- 904.4/904.4 kB 7.2 MB/s eta 0:00:00
     ---------------------------------------- 0.0/199.7 kB ? eta -:--:--
     ---------------------------------------- 199.7/199.7 kB ? eta 0:00:00
  Using cached contourpy-1.1.1-cp38-cp38-win_amd64.whl.metadata (5.9 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.47.2-cp38-cp38-win_amd64.whl.metadata (160 kB)
  Using cached kiwisolver-1.4.5-cp38-cp38-win_amd64.whl.metadata (6.5 kB)
  Using cached pillow-10.2.0-cp38-cp38-win_amd64.whl.metadata (9.9 kB)
  Using cached pyparsing-3.1.1-py3-none-any.whl.metadata (5.1 kB)
   ----------------------------------------

In [1]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [25]:
# open picture and represent it as an array
frame_sample = cv2.imread('../images/sample_002.jpg')
# downscale the frame to 480x640
frame_sample = cv2.resize(frame_sample, (640, 480))
print(type(frame_sample), frame_sample.shape)

<class 'numpy.ndarray'> (480, 640, 3)


In [15]:
# VIDEO FEED
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow('Mediapipe Feed', frame_sample)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [8]:
print(type(frame), frame.shape)

<class 'numpy.ndarray'> (480, 640, 3)


# Make detection

In [29]:
cap = cv2.VideoCapture(0)
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor image to RGB
        image = cv2.cvtColor(frame_sample, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                  )

        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()